## OmniFold

In [1]:
from unfold import *

import os
os.environ['CUDA_VISIBLE_DEVICES']="1"

import tensorflow as tf
import tensorflow.keras.backend as K
tf.random.set_seed(1234)
np.random.seed(1234)

import numpy as np
import matplotlib.pyplot as plt
import energyflow as ef
import energyflow.archs
from energyflow.archs import PFN
from matplotlib import gridspec

from matplotlib import rc
rc('font', size=20)

physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [2]:
iterations = 5
dims = 2

In [3]:
datasets = {'Pythia26': ef.zjets_delphes.load('Pythia26', num_data=10**6),
            'Herwig': ef.zjets_delphes.load('Herwig', num_data=10**6)}

<__array_function__ internals>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [4]:
pT_true = datasets['Pythia26']['gen_jets'][:,0]
m_true = datasets['Pythia26']['gen_jets'][:,3]
pT_reco = datasets['Pythia26']['sim_jets'][:,0]
m_reco = datasets['Pythia26']['sim_jets'][:,3]

pT_true_alt = datasets['Herwig']['gen_jets'][:,0]
m_true_alt = datasets['Herwig']['gen_jets'][:,3]
pT_reco_alt = datasets['Herwig']['sim_jets'][:,0]
m_reco_alt = datasets['Herwig']['sim_jets'][:,3]

In [ ]:
weights, models, history = multifold(dims, iterations, np.c_[pT_true/100.-2,m_true/10.-2], np.c_[pT_reco/100.-2,m_reco/10.-2], np.c_[pT_reco_alt/100.-2,m_reco_alt/10.-2], verbose=1)

ITERATION: 1
STEP 1...
Epoch 1/1000
7500/7500 [==============================] - 28s 4ms/step - loss: 0.6850 - accuracy: 0.5051 - val_loss: 0.6848 - val_accuracy: 0.4925
Epoch 2/1000
7500/7500 [==============================] - 27s 4ms/step - loss: 0.6848 - accuracy: 0.5114 - val_loss: 0.6846 - val_accuracy: 0.5221
Epoch 3/1000
7500/7500 [==============================] - 26s 3ms/step - loss: 0.6847 - accuracy: 0.5110 - val_loss: 0.6847 - val_accuracy: 0.5304
Epoch 4/1000
7500/7500 [==============================] - 26s 3ms/step - loss: 0.6847 - accuracy: 0.5115 - val_loss: 0.6846 - val_accuracy: 0.5220
Epoch 5/1000
7500/7500 [==============================] - 27s 4ms/step - loss: 0.6847 - accuracy: 0.5112 - val_loss: 0.6848 - val_accuracy: 0.4970
Epoch 6/1000
7500/7500 [==============================] - 27s 4ms/step - loss: 0.6847 - accuracy: 0.5131 - val_loss: 0.6846 - val_accuracy: 0.5032
Epoch 7/1000
7500/7500 [==============================] - 26s 4ms/step - loss: 0.6846 - accurac

7500/7500 [==============================] - 25s 3ms/step - loss: 0.6951 - accuracy: 0.4169 - val_loss: 0.6952 - val_accuracy: 0.4271
Epoch 15/1000
7500/7500 [==============================] - 27s 4ms/step - loss: 0.6951 - accuracy: 0.4171 - val_loss: 0.6952 - val_accuracy: 0.4171
Epoch 16/1000
7500/7500 [==============================] - 26s 3ms/step - loss: 0.6951 - accuracy: 0.4160 - val_loss: 0.6952 - val_accuracy: 0.4096
Epoch 17/1000
7500/7500 [==============================] - 27s 4ms/step - loss: 0.6951 - accuracy: 0.4172 - val_loss: 0.6951 - val_accuracy: 0.4080
Epoch 18/1000
7500/7500 [==============================] - 27s 4ms/step - loss: 0.6951 - accuracy: 0.4145 - val_loss: 0.6951 - val_accuracy: 0.4215
Epoch 19/1000
7500/7500 [==============================] - 26s 4ms/step - loss: 0.6951 - accuracy: 0.4181 - val_loss: 0.6952 - val_accuracy: 0.4139
Epoch 20/1000
7500/7500 [==============================] - 26s 3ms/step - loss: 0.6951 - accuracy: 0.4165 - val_loss: 0.6952 -

In [ ]:
    for i in range(iterations):
    fig = plt.figure(figsize=(8, 6)) 
    gs = gridspec.GridSpec(1, 1, height_ratios=[1]) 
    ax0 = plt.subplot(gs[0])
    ax0.yaxis.set_ticks_position('both')
    ax0.xaxis.set_ticks_position('both')
    ax0.tick_params(direction="in",which="both")
    ax0.minorticks_on()
    plt.xticks(fontsize=20)
    plt.yticks(fontsize=20)

    OF,_,_  = plt.hist(m_true_alt,bins=np.linspace(0,70,30),color="blue",label="Target",density=True,alpha=0.2)
    T,_,_  = plt.hist(m_true,bins=np.linspace(0,70,30),ls=":",histtype="step",color="black",lw=3,label="Prior",density=True)
    G,_,_ = plt.hist(m_true,weights=weights[i,1],bins=np.linspace(0,70,30),lw=3,histtype="step",ls="-",color="black",label="OmniFold (it. "+str(i)+")",density=True)
    plt.legend()
    plt.xlabel("Truth jet mass [GeV]")
    plt.ylabel("Normalized to unity")

    fig.savefig('figures/OmniFold_iteration_'+str(i)+'.pdf',bbox_inches='tight')

In [ ]:
myweights = of.omnifold(theta0,theta_unknown_S,2,model)

In [ ]:
_,_,_=plt.hist(theta0_G,bins = bins,color='blue',alpha=0.5,label="MC, true")
_,_,_=plt.hist(theta_unknown_G,bins = bins,color='orange',alpha=0.5,label="Data, true")
_,_,_=plt.hist(theta0_G,weights=myweights[-1, 0, :], bins = bins,color='black',histtype="step",label="OmniFolded",lw="2")
plt.xlabel("x")
plt.ylabel("events")
plt.legend(frameon=False)

In [ ]:
print("Moment predictions:")
print("Gen Mean:", np.average(theta0_G))
print("Truth Mean:", np.average(theta_unknown_G))
print("Unfolded Gen Mean:", np.average(theta0_G,weights=myweights[-1, 0, :]))

In [ ]:
1+1